# Noisy OR logic gate

The Noisy OR gate is based on the assumption that each input can independently cause the output. The inputs could be termed 'causes' and the output, the 'effect'.

A two-input Noisy OR gate with two inputs $a$ and $b$ is parameterised with $p_a$ and $p_b$. The definition of each probability is that it is the probability of the effect given that the cause occurs.

In [1]:
import numpy as np
import pandas as pd

In [2]:
p_a = 0.4
p_b = 0.6

n_experiments_per_configuration = 10000

def sample(a, b, p_a, p_b):
    a_prime = 0
    if a == 1:
        a_prime = np.random.binomial(1, p_a, size=None)
    
    b_prime = 0
    if b == 1:
        b_prime = np.random.binomial(1, p_b, size=None)
    
    return a_prime or b_prime

def prob(a, b, p_a, p_b, n_experiments):
    samples = [sample(a, b, p_a, p_b) for _ in range(n_experiments)]
    return sum(samples) / n_experiments

In [3]:
df = pd.DataFrame({"a":[0,0,1,1], 
                   "b": [0,1,0,1], 
                   "experimental": [None, None, None, None], 
                   "predicted": [None, None, None, None]})

for row_idx in range(len(df)):
    a = df["a"][row_idx]
    b = df["b"][row_idx]
    df.loc[row_idx, "experimental"] = prob(a, b, p_a, p_b, n_experiments_per_configuration)
    df.loc[row_idx, "predicted"] = 1 - ((1-p_a)**a) * ((1-p_b)**b)
    
df

,a,b,experimental,predicted
0,0,0,0.0000,0.00
1,0,1,0.5971,0.60
2,1,0,0.3918,0.40
3,1,1,0.7620,0.76


## Inserting an extra node

### Checking the mathematical expression

In [4]:
y_given_a_b = {
    (0, 0): 0.2,
    (0, 1): 0.3,
    (1, 0): 0.4,
    (1, 1): 0.6
}

x_given_y_c = {
    (0, 0): 0.25,
    (0, 1): 0.35,
    (1, 0): 0.45,
    (1, 1): 0.55
}

In [5]:
x_given_a_b_c = {}

for a in [0,1]:
    for b in [0,1]:
        for c in [0,1]:
            x_given_a_b_c[(a,b,c)] = (1-y_given_a_b[(a,b)])*x_given_y_c[(0,c)] + y_given_a_b[(a,b)]*x_given_y_c[(1,c)]

In [6]:
x_given_a_b_c

{(0, 0, 0): 0.29000000000000004,
 (0, 0, 1): 0.39,
 (0, 1, 0): 0.31,
 (0, 1, 1): 0.41,
 (1, 0, 0): 0.33,
 (1, 0, 1): 0.43000000000000005,
 (1, 1, 0): 0.37,
 (1, 1, 1): 0.47}

### Gate inbetween Noisy OR

In [7]:
from scipy.optimize import minimize, shgo

In [8]:
def l1_norm(x):
    return np.sum(np.abs(x))

In [9]:
def l2_norm(x):
    return np.sqrt(np.sum(np.power(x, 2)))

In [10]:
print(l1_norm(np.array([1,2,3])))
print(l2_norm(np.array([1,2,3])))

6
3.7416573867739413


In [11]:
def make_error_fn(pa, pb, pc, norm):
    assert norm in ["l1-norm", "l2-norm"]

    def total_error(probs):
        assert len(probs) == 8
        p = probs[:4]
        lam = probs[4:]

        e000 = (1-p[0])*lam[0] + p[0]*lam[2] - 0
        e001 = (1-p[0])*lam[1] + p[0]*lam[3] - pc
        e010 = (1-p[1])*lam[0] + p[1]*lam[2] - pb
        e011 = (1-p[1])*lam[1] + p[1]*lam[3] - (1-(1-pb)*(1-pc))
        e100 = (1-p[2])*lam[0] + p[2]*lam[2] - pa
        e101 = (1-p[2])*lam[1] + p[2]*lam[3] - (1-(1-pa)*(1-pc))
        e110 = (1-p[3])*lam[0] + p[3]*lam[2] - (1-(1-pa)*(1-pb))
        e111 = (1-p[3])*lam[1] + p[3]*lam[3] - (1-(1-pa)*(1-pb)*(1-pc))

        e = np.array([e000, e001, e010, e011, e100, e101, e110, e111])
        
        if norm == "l1-norm":
            return l1_norm(e)
        else:
            return l2_norm(e)

    return total_error

In [12]:
pa = 0.2
pb = 0.3
pc = 0.5
f = make_error_fn(pa, pb, pc, "l2-norm")

In [13]:
def make_prob_bounds(n):
    return tuple([(0,1) for _ in range(n)])

Optimisation methods available in Scipy that are applicable:

* Powell -- 
* L-BFGS-B --
* TNC -- 
* SLSQP -- 
* trust-constr -- 

If there is a Jacobian:

* Newton-CG -- requires Jacobian
* trust-exact -- requires Jacobian
* trust-krylov -- requires Jacobian

Non-applicable methods:

* Nelder-Mead -- doesn't handle bounds
* CG -- doesn't handle bounds
* BFGS -- doesn't handle bounds
* COBYLA -- doesn't handle bounds
* dogleg -- doesn't handle bounds
* trust-ncg -- doesn't handle bounds

In [14]:
initial = np.repeat(0.5, 8)
bounds = make_prob_bounds(8)

# Minimize 
result = minimize(f, x0=initial, method='Powell', bounds=bounds, options={'ftol': 1e-9, 'disp': True})
print(result)

Optimization terminated successfully.
         Current function value: 0.000069
         Iterations: 14
         Function evaluations: 1958
   direc: array([[-5.52924306e-20,  0.00000000e+00, -8.26194852e-21,
         0.00000000e+00, -1.20488171e-36, -1.08405883e-18,
         7.07931593e-21,  1.14292052e-20],
       [-4.96019009e-04, -8.13237211e-05, -9.93900586e-05,
        -1.56043035e-04,  0.00000000e+00, -8.81860581e-05,
         7.52855669e-05,  1.48908996e-04],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.

In [15]:
result.x

array([1.98551965e-08, 6.20264968e-01, 4.13470010e-01, 9.09793724e-01,
       6.61069614e-05, 5.00007184e-01, 4.83619449e-01, 7.41810946e-01])

In [16]:
f(result.x)

6.887100656327051e-05

In [17]:
p = result.x[:4]
lam = result.x[4:]

print(" y | c  | x")
print(f"---|----|-------")
print(f" 0 | 0  | {lam[0]}")
print(f" 0 | 1  | {lam[1]}")
print(f" 1 | 0  | {lam[2]}")
print(f" 1 | 1  | {lam[3]}")
print("")
print(" a | b  | y")
print(f"---|----|-------")
print(f" 0 | 0  | {p[0]}")
print(f" 0 | 1  | {p[1]}")
print(f" 1 | 0  | {p[2]}")
print(f" 1 | 1  | {p[3]}")

 y | c  | x
---|----|-------
 0 | 0  | 6.610696135189607e-05
 0 | 1  | 0.5000071835066979
 1 | 0  | 0.4836194487845294
 1 | 1  | 0.7418109462653955

 a | b  | y
---|----|-------
 0 | 0  | 1.9855196508610543e-08
 0 | 1  | 0.6202649681822469
 1 | 0  | 0.41347000992687627
 1 | 1  | 0.909793724319774


In [18]:
def show_p_x(result, pa, pb, pc):
    p = result.x[:4]
    lam = result.x[4:]

    e000 = (1-p[0])*lam[0] + p[0]*lam[2]
    e001 = (1-p[0])*lam[1] + p[0]*lam[3]
    e010 = (1-p[1])*lam[0] + p[1]*lam[2]
    e011 = (1-p[1])*lam[1] + p[1]*lam[3]
    e100 = (1-p[2])*lam[0] + p[2]*lam[2]
    e101 = (1-p[2])*lam[1] + p[2]*lam[3]
    e110 = (1-p[3])*lam[0] + p[3]*lam[2]
    e111 = (1-p[3])*lam[1] + p[3]*lam[3]  
    
    print("| a | b | c | p(x|a,b,c)                | Noisy OR                  |")
    print("|---|---|---|---------------------------|---------------------------|")
    print(f"| 0 | 0 | 0 | {e000:<25} | {0:<25} |")
    print(f"| 0 | 0 | 1 | {e001:<25} | {pc:<25} |")
    print(f"| 0 | 1 | 0 | {e010:<25} | {pb:<25} |")
    print(f"| 0 | 1 | 1 | {e011:<25} | {(1-(1-pb)*(1-pc)):<25} |")
    print(f"| 1 | 0 | 0 | {e100:<25} | {pa:<25} |")
    print(f"| 1 | 0 | 1 | {e101:<25} | {(1-(1-pa)*(1-pc)):<25} |")
    print(f"| 1 | 1 | 0 | {e110:<25} | {(1-(1-pa)*(1-pb)):<25} |")
    print(f"| 1 | 1 | 1 | {e111:<25} | {(1-(1-pa)*(1-pb)*(1-pc)):<25} |")

In [19]:
show_p_x(result, pa, pb, pc)

| a | b | c | p(x|a,b,c)                | Noisy OR                  |
|---|---|---|---------------------------|---------------------------|
| 0 | 0 | 0 | 6.611656239852037e-05     | 0                         |
| 0 | 0 | 1 | 0.5000071883077591        | 0.5                       |
| 0 | 1 | 0 | 0.29999730514172424       | 0.3                       |
| 0 | 1 | 1 | 0.649989586720569         | 0.65                      |
| 1 | 0 | 0 | 0.2000009120051553        | 0.2                       |
| 1 | 0 | 1 | 0.5999857876948926        | 0.6                       |
| 1 | 1 | 0 | 0.43999990272593326       | 0.44000000000000006       |
| 1 | 1 | 1 | 0.7199987293814684        | 0.72                      |


The Noisy OR can be approximated very closely with an extra node capturing dependencies between $a$ and $b$.

## Weighted function fitting

In [20]:
def knowns_to_vector(knowns):
    """Convert a dict of known probabilities (from users) to an array."""
    
    # Preconditions
    assert type(knowns) == dict
    
    lookup = {
        "p0": 0,
        "pc": 1,
        "pb": 2,
        "pbc": 3,
        "pa": 4,
        "pac": 5,
        "pab": 6,
        "pabc": 7
    }    

    defined = [False for _ in range(8)]
    p = np.zeros(8)
    
    for name,prob in knowns.items():
        idx = lookup[name]
        p[idx] = prob
        defined[idx] = True
    
    return p, defined

# Tests
knowns_to_vector({"pab": 1})

(array([0., 0., 0., 0., 0., 0., 1., 0.]),
 [False, False, False, False, False, False, True, False])

In [21]:
def make_error_fn(knowns, weight):
    
    def total_error(probs):
        
        assert len(probs) == 8
        p = probs[:4]
        lam = probs[4:]

        p_added_node = np.array([
            (1-p[0])*lam[0] + p[0]*lam[2],  # 000
            (1-p[0])*lam[1] + p[0]*lam[3],  # 001
            (1-p[1])*lam[0] + p[1]*lam[2],  # 010
            (1-p[1])*lam[1] + p[1]*lam[3],  # 011
            (1-p[2])*lam[0] + p[2]*lam[2],  # 100
            (1-p[2])*lam[1] + p[2]*lam[3],  # 101
            (1-p[3])*lam[0] + p[3]*lam[2],  # 110
            (1-p[3])*lam[1] + p[3]*lam[3]   # 111
        ])
        
        pa = knowns["pa"]
        pb = knowns["pb"]
        pc = knowns["pc"]
        
        p_noisy_or = np.array([
            0,                       # 000
            pc,                      # 001
            pb,                      # 010
            (1-(1-pb)*(1-pc)),       # 011
            pa,                      # 100
            (1-(1-pa)*(1-pc)),       # 101
            (1-(1-pa)*(1-pb)),       # 110
            (1-(1-pa)*(1-pb)*(1-pc)) # 111
        ])

        p_user, defined = knowns_to_vector(knowns)

        # Error terms
        e = [None for _ in range(8)]
        weights = [None for _ in range(8)]
        
        for i in range(8):
            if defined[i]:
                # Use user-defined probability
                e[i] = p_added_node[i] - p_user[i]
                weights[i] = weight
            else:
                # Use Noisy OR
                e[i] = p_added_node[i] - p_noisy_or[i]
                weights[i] = 1
    
        e = np.array(e)
        weights = np.array(weights)
        
        # Return the weighted squared error
        return np.sum(weights * np.power(e, 2))


    return total_error

In [22]:
knowns = {"p0": 0, "pa": 0.2, "pb": 0.3, "pc": 0.4, "pab": 0.7, "pbc": 0.4, "pabc": 0.2}
weight = 1
f = make_error_fn(knowns, weight)

In [23]:
[True, True, True, False, True, False, True, False]

[True, True, True, False, True, False, True, False]

In [24]:
initial = np.repeat(0.5, 8)
bounds = make_prob_bounds(8)

# Minimize 
result = minimize(f, x0=initial, method='Powell', bounds=bounds, options={'ftol': 1e-9, 'disp': True})
print(result)

Optimization terminated successfully.
         Current function value: 0.017847
         Iterations: 12
         Function evaluations: 925
   direc: array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: 0.017846905484085546
 message: 'Optimization terminated successfully.'
    nfev: 925
     nit: 12
  status: 0
 success: True
       x: array([0.02574094, 0.40237485, 0.21959348, 0.99994697, 0.01328336,
       0.48893063, 0.70934283, 0.22448057])


In [25]:
def noisy_or_probabilities(pa, pb, pc):
    return np.array([
        0.0,
        pc,
        pb,
        1-(1-pb)*(1-pc),
        pa,
        1-(1-pa)*(1-pc),
        1-(1-pa)*(1-pb),
        1-(1-pa)*(1-pb)*(1-pc)
    ])

In [26]:
def show_p_x(result, knowns):
    p = result.x[:4]
    lam = result.x[4:]
    
    pa = knowns["pa"]
    pb = knowns["pb"]
    pc = knowns["pc"]

    p_user, defined = knowns_to_vector(knowns)
    p_user_str = [str(p_user[i]) if defined[i] else "" for i in range(8)]
    
    # Actual probabilities given the extra node
    p_added_node = np.array([
        (1-p[0])*lam[0] + p[0]*lam[2],  # 000
        (1-p[0])*lam[1] + p[0]*lam[3],  # 001
        (1-p[1])*lam[0] + p[1]*lam[2],  # 010
        (1-p[1])*lam[1] + p[1]*lam[3],  # 011
        (1-p[2])*lam[0] + p[2]*lam[2],  # 100
        (1-p[2])*lam[1] + p[2]*lam[3],  # 101
        (1-p[3])*lam[0] + p[3]*lam[2],  # 110
        (1-p[3])*lam[1] + p[3]*lam[3]   # 111
    ])
    
    noisy_or = noisy_or_probabilities(pa, pb, pc)
    
    print("| a | b | c | p(x|a,b,c)                | p(user)                   | Noisy OR                  |")
    print("|---|---|---|---------------------------|---------------------------|---------------------------|")
    print(f"| 0 | 0 | 0 | {p_added_node[0]:<25} | {p_user_str[0]:<25} | {noisy_or[0]:<25} |")
    print(f"| 0 | 0 | 1 | {p_added_node[1]:<25} | {p_user_str[1]:<25} | {noisy_or[1]:<25} |")
    print(f"| 0 | 1 | 0 | {p_added_node[2]:<25} | {p_user_str[2]:<25} | {noisy_or[2]:<25} |")
    print(f"| 0 | 1 | 1 | {p_added_node[3]:<25} | {p_user_str[3]:<25} | {noisy_or[3]:<25} |")
    print(f"| 1 | 0 | 0 | {p_added_node[4]:<25} | {p_user_str[4]:<25} | {noisy_or[4]:<25} |")
    print(f"| 1 | 0 | 1 | {p_added_node[5]:<25} | {p_user_str[5]:<25} | {noisy_or[5]:<25} |")
    print(f"| 1 | 1 | 0 | {p_added_node[6]:<25} | {p_user_str[6]:<25} | {noisy_or[6]:<25} |")
    print(f"| 1 | 1 | 1 | {p_added_node[7]:<25} | {p_user_str[7]:<25} | {noisy_or[7]:<25} |")
    
    error_p_user = 0.0
    error_noisy_or = 0.0
    
    for i in range(8):
        if defined[i]:
            error_p_user += np.power(p_added_node[i] - p_user[i], 2)
        else:
            error_noisy_or += np.power(noisy_or[i] - p_user[i], 2)
            
    print(f"Total squared error for user: {error_p_user}")
    print(f"Total squared error for Noisy OR: {error_noisy_or}")

In [27]:
show_p_x(result, knowns)

| a | b | c | p(x|a,b,c)                | p(user)                   | Noisy OR                  |
|---|---|---|---------------------------|---------------------------|---------------------------|
| 0 | 0 | 0 | 0.031200591362602642      | 0.0                       | 0.0                       |
| 0 | 0 | 1 | 0.4821234352304321        | 0.4                       | 0.4                       |
| 0 | 1 | 0 | 0.2933601889703703        | 0.3                       | 0.3                       |
| 0 | 1 | 1 | 0.38252257651322896       | 0.4                       | 0.5800000000000001        |
| 1 | 0 | 0 | 0.16613348148401527       | 0.2                       | 0.2                       |
| 1 | 0 | 1 | 0.4308591220108405        |                           | 0.52                      |
| 1 | 1 | 0 | 0.7093059203004298        | 0.7                       | 0.44000000000000006       |
| 1 | 1 | 1 | 0.22449459509172615       | 0.2                       | 0.664                     |
Total squared error 

In [28]:
p = result.x[:4]
lam = result.x[4:]

print(" y | c  | x")
print(f"---|----|-------")
print(f" 0 | 0  | {lam[0]}")
print(f" 0 | 1  | {lam[1]}")
print(f" 1 | 0  | {lam[2]}")
print(f" 1 | 1  | {lam[3]}")
print("")
print(" a | b  | y")
print(f"---|----|-------")
print(f" 0 | 0  | {p[0]}")
print(f" 0 | 1  | {p[1]}")
print(f" 1 | 0  | {p[2]}")
print(f" 1 | 1  | {p[3]}")

 y | c  | x
---|----|-------
 0 | 0  | 0.0132833634954359
 0 | 1  | 0.4889306293888226
 1 | 0  | 0.7093428315283126
 1 | 1  | 0.22448057161127008

 a | b  | y
---|----|-------
 0 | 0  | 0.025740944114735417
 0 | 1  | 0.4023748520603496
 1 | 0  | 0.21959347585709424
 1 | 1  | 0.9999469711575262


## An approach assuming dependencies

In [29]:
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from IPython.display import clear_output

import matplotlib.pyplot as plt

In [30]:
pa_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.01, description="pa")
pb_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.01, description="pb")
pc_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.01, description="pc")

override_pab = widgets.ToggleButton(
    value=False,
    description='Override pab',
    disabled=False,
    button_style='info',
    tooltip='Description',
    icon='check'
)
pab_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.01, description="pab")

override_pac = widgets.ToggleButton(
    value=False,
    description='Override pac',
    disabled=False,
    button_style='info',
    tooltip='Description',
    icon='check'
)
pac_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.01, description="pac")

override_pbc = widgets.ToggleButton(
    value=False,
    description='Override pbc',
    disabled=False,
    button_style='info',
    tooltip='Description',
    icon='check'
)
pbc_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.01, description="pbc")

In [31]:
def plot(pa, pb, pc, override_pab, pab, override_pac, pac, override_pbc, pbc):  
    
    def add_point(ps):
        t = 1.0
        for p in ps:
            t *= (1-p)
        plt.plot(1-t, 3, 'g.')
    
    # ONE input ----
    
    #100
    plt.plot(pa, 9, 'rx')   
    
    #010
    plt.plot(pb, 8, 'rx')
    
    #001
    plt.plot(pc, 7, 'rx')
    
    plt.plot([-0.1, 1.1], [6.5, 6.5], 'k--')
    
    # TWO inputs ----
    
    #110
    plt.plot(1-(1-pa)*(1-pb), 6, 'rx')
    if override_pab:
        plt.plot(pab, 6, 'gs')     

    #101
    plt.plot(1-(1-pa)*(1-pc), 5, 'rx')
    if override_pac:
        plt.plot(pac, 5, 'gs')    
    
    #011
    plt.plot(1-(1-pb)*(1-pc), 4, 'rx')
    if override_pbc:
        plt.plot(pbc, 4, 'gs')
        
    plt.plot([-0.1, 1.1], [3.5, 3.5], 'k--')    
    
    # THREE inputs ----

    #111
    plt.plot(1-(1-pa)*(1-pb)*(1-pc), 3, 'rx')    
    
    v = [override_pab, override_pac, override_pbc]
    print(v)
    if v == [False, False, True]:
        add_point([pa, pbc])
    elif v == [False, True, False]:
        add_point([pac, pb])
    elif v == [False, True, True]:
        add_point([pab, pbc])
        add_point([pb, pac])
        add_point([pa, pbc])
    elif v == [True, False, False]:
        add_point([pab, pc])
    elif v == [True, False, True]:
        add_point([pab, pc])
        add_point([pbc, pa])
        add_point([pab, bc])
    elif v == [True, True, False]:
        add_point([pab, pc])
        add_point([pac, pb])
        add_point([pab, pac])
    elif v == [True, True, True]:
        #add_point([pa, pb, pc])
        add_point([pa, pbc])
        add_point([pb, pac])        
        add_point([pc, pab])
        #add_point([pac, pb])
        #add_point([pbc, pa])
        #add_point([pab, pac])
        #add_point([pab, pbc])
        #add_point([pac, pbc])
        
    plt.yticks([3, 4, 5, 6, 7, 8, 9], ["abc", "bc", "ac", "ab", "c", "b", "a"])
    plt.xlim(-0.1, 1.1)
    plt.ylim(0, 9.5)
    plt.show()

In [33]:
interactive(plot, pa=pa_slider, pb=pb_slider, pc=pc_slider, 
            override_pab=override_pab, pab=pab_slider,
            override_pac=override_pac, pac=pac_slider,
            override_pbc=override_pbc, pbc=pbc_slider,
            continuous_update=False)

interactive(children=(FloatSlider(value=0.1, description='pa', max=1.0, step=0.01), FloatSlider(value=0.41, de…